In [1]:
import pandas as pd
import duckdb as ddb

## Data Extraction and Aggregation

Due to the volume of data found in `Data.csv` I found it easier to operate on the data using DuckDB. At a high level I wanted a location from which I could easily extract data as and when I needed, as well as store results without flooding my RAM.

The steps taken are:
- Creating a DuckDB instance
- Importing data from the `Data.csv`, `Kalam Climate Data.xlsx` and `SampleSubmission.csv` files
- Aggregate the raw data (hydropower data and climate data) into a daily granularity tables
- Use the database as and when needed to store data

In [ ]:
# Create the database and two schemas
con = ddb.connect("./kalam_hydropower.db")

try:
    con.sql("create schema 'raw';")
    con.sql("create schema 'prepared';")
except ddb.CatalogException as e:
    print(f"Schemas already exist.")

In [3]:
# Check the datatypes etc before loading - note that v_blue and v_yellow are detected as varchars and must be explicitly cast to doubles
con.sql("select * from read_csv('./data/Data.csv') limit 10;")

┌─────────────────────┬────────┬─────────┬──────────────┬────────────────────┬────────────────────────────────┬─────────┬──────────┬───────────────────┬───────────────────┐
│      date_time      │ v_red  │ current │ power_factor │        kwh         │             Source             │ v_blue  │ v_yellow │ consumer_device_9 │ consumer_device_x │
│      timestamp      │ double │ double  │    double    │       double       │            varchar             │ varchar │ varchar  │       int64       │       int64       │
├─────────────────────┼────────┼─────────┼──────────────┼────────────────────┼────────────────────────────────┼─────────┼──────────┼───────────────────┼───────────────────┤
│ 2024-07-22 18:20:00 │ 137.65 │    0.08 │         0.72 │         0.00066072 │ consumer_device_10_data_user_1 │ NULL    │ NULL     │                 0 │                10 │
│ 2024-07-22 18:25:00 │ 122.82 │    0.08 │         0.73 │ 0.0005977239999999 │ consumer_device_10_data_user_1 │ NULL    │ NULL     │   

In [4]:
# Create a table ignoring the columns consumer device 9 etc - these are assumed to be errors
try:
    con.sql("""
        create table raw.hydropower_production as
            select date_time, Source as source, v_red, cast(v_blue as double) as v_blue, cast(v_yellow as double) as v_yellow, current, kwh
            from read_csv('./data/Data.csv');
    """)
except ddb.CatalogException as e:
    print(f"Table already exists: {e}")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [5]:
# This is a package needed to read excel natively in DDB
con.sql("INSTALL excel; LOAD excel")

con.sql("select * from './data/Climate Data/Kalam Climate Data.xlsx' limit 10")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌─────────────────────┬──────────────────┬───────────────────────────┬────────────────────────┬────────────────────────┬──────────────────────────┬───────────────┬────────────────┐
│      Date Time      │ Temperature (°C) │ Dewpoint Temperature (°C) │ U Wind Component (m/s) │ V Wind Component (m/s) │ Total Precipitation (mm) │ Snowfall (mm) │ Snow Cover (%) │
│      timestamp      │      double      │          double           │         double         │         double         │          double          │    double     │     double     │
├─────────────────────┼──────────────────┼───────────────────────────┼────────────────────────┼────────────────────────┼──────────────────────────┼───────────────┼────────────────┤
│ 2023-06-03 13:00:00 │      7.199731445 │              -2.448126221 │           -0.034194946 │             0.06098938 │                 0.000377 │           0.0 │    99.97265625 │
│ 2023-06-03 14:00:00 │      6.000619507 │              -2.597540283 │           -0.080688477 │

In [6]:
# Rename climate column names as the unusual characters make things difficult
try:
    con.sql("""
        create table raw.climate as
            select "Date Time" as date_time, "Temperature (°C)" as temperature, "Dewpoint Temperature (°C)" as dewpoint_temperature, "U Wind Component (m/s)" as u_wind_component, "V Wind Component (m/s)" as v_wind_component, "Total Precipitation (mm)" as total_precipitation, "Snowfall (mm)" as snowfall, "Snow Cover (%)" as snow_cover_perc
            from './data/Climate Data/Kalam Climate Data.xlsx'
    """)
except ddb.CatalogException as e:
    print(f"Table already exists: {e}")

In [7]:
con.sql("select * from raw.climate limit 10")

┌─────────────────────┬──────────────┬──────────────────────┬──────────────────┬──────────────────┬─────────────────────┬──────────┬─────────────────┐
│      date_time      │ temperature  │ dewpoint_temperature │ u_wind_component │ v_wind_component │ total_precipitation │ snowfall │ snow_cover_perc │
│      timestamp      │    double    │        double        │      double      │      double      │       double        │  double  │     double      │
├─────────────────────┼──────────────┼──────────────────────┼──────────────────┼──────────────────┼─────────────────────┼──────────┼─────────────────┤
│ 2023-06-03 13:00:00 │  7.199731445 │         -2.448126221 │     -0.034194946 │       0.06098938 │            0.000377 │      0.0 │     99.97265625 │
│ 2023-06-03 14:00:00 │  6.000619507 │         -2.597540283 │     -0.080688477 │      0.018585205 │            0.000398 │      0.0 │     99.97265625 │
│ 2023-06-03 15:00:00 │  4.789971924 │         -2.696951294 │      0.057449341 │     -0.352020

In [8]:
try:
    con.sql("""
        create table raw.sample_submission as
            select * from read_csv('./data/SampleSubmission.csv')
    """)
except ddb.CatalogException as e:
    print(f"Table already exists: {e}")

In [9]:
con.sql("select * from raw.sample_submission limit 10")

┌───────────────────────────────────────────┬───────┐
│                    ID                     │  kwh  │
│                  varchar                  │ int64 │
├───────────────────────────────────────────┼───────┤
│ 2024-09-24_consumer_device_12_data_user_1 │     0 │
│ 2024-09-25_consumer_device_12_data_user_1 │     0 │
│ 2024-09-26_consumer_device_12_data_user_1 │     0 │
│ 2024-09-27_consumer_device_12_data_user_1 │     0 │
│ 2024-09-28_consumer_device_12_data_user_1 │     0 │
│ 2024-09-29_consumer_device_12_data_user_1 │     0 │
│ 2024-09-30_consumer_device_12_data_user_1 │     0 │
│ 2024-10-01_consumer_device_12_data_user_1 │     0 │
│ 2024-10-02_consumer_device_12_data_user_1 │     0 │
│ 2024-10-03_consumer_device_12_data_user_1 │     0 │
├───────────────────────────────────────────┴───────┤
│ 10 rows                                 2 columns │
└───────────────────────────────────────────────────┘

### Aggregation
The main goal here is to make the tables `raw.hydropower_production` and `raw.climate` exist at a daily granularity, this is easier to work with in pandas and the forecasting exercise/validation occurs on a daily level rather than at 5 minute or 1 hour intervals.

In [10]:
# Aggregate the power production data
try:
    con.sql("""
    create table prepared.daily_hydropower_production as
            with temp as (
                select
                    *,
                    cast(date_time as date) as date,
                    regexp_extract(source, 'consumer_device_(\d+)', 1) as consumer_device,
                    regexp_extract(source, '_data_user_(\d+)', 1) as data_user
                from raw.hydropower_production
            )
            select
                date,
                source,
                consumer_device,
                data_user,
                sum(kwh) as kwh
            from temp
            group by date, source, consumer_device, data_user
            order by source, date
    """)
except ddb.CatalogException as e:
    print(f"Table already exists: {e}")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [11]:
con.sql("select * from prepared.daily_hydropower_production limit 10")

┌────────────┬────────────────────────────────┬─────────────────┬───────────┬──────────────────────┐
│    date    │             source             │ consumer_device │ data_user │         kwh          │
│    date    │            varchar             │     varchar     │  varchar  │        double        │
├────────────┼────────────────────────────────┼─────────────────┼───────────┼──────────────────────┤
│ 2024-07-22 │ consumer_device_10_data_user_1 │ 10              │ 1         │   0.0243301524166643 │
│ 2024-07-23 │ consumer_device_10_data_user_1 │ 10              │ 1         │  0.10355964899999259 │
│ 2024-07-24 │ consumer_device_10_data_user_1 │ 10              │ 1         │  0.13754253833332433 │
│ 2024-07-25 │ consumer_device_10_data_user_1 │ 10              │ 1         │  0.12101083841665729 │
│ 2024-07-26 │ consumer_device_10_data_user_1 │ 10              │ 1         │                  0.0 │
│ 2024-07-27 │ consumer_device_10_data_user_1 │ 10              │ 1         │              

In [12]:
# Similarly aggregate the climate data
try:
    con.sql("""
        create table prepared.daily_climate as (
        select
            cast(date_time as date) as date,
            avg(temperature) as avg_temperature,
            avg(dewpoint_temperature) as avg_dewpoint_temperature,
            avg(u_wind_component) as avg_u_wind_component,
            avg(v_wind_component) as avg_v_wind_component,
            sum(total_precipitation) as total_precipitation,
            sum(snowfall) as total_snowfall,
            avg(snow_cover_perc) as avg_snow_cover_perc
        from raw.climate
        group by cast(date_time as date)
    )
    """)
except ddb.CatalogException as e:
    print(f"Table already exists: {e}")

In [13]:
con.sql("select * from prepared.daily_climate limit 10")

┌────────────┬────────────────────┬──────────────────────────┬───────────────────────┬──────────────────────┬───────────────────────┬────────────────────────┬─────────────────────┐
│    date    │  avg_temperature   │ avg_dewpoint_temperature │ avg_u_wind_component  │ avg_v_wind_component │  total_precipitation  │     total_snowfall     │ avg_snow_cover_perc │
│    date    │       double       │          double          │        double         │        double        │        double         │         double         │       double        │
├────────────┼────────────────────┼──────────────────────────┼───────────────────────┼──────────────────────┼───────────────────────┼────────────────────────┼─────────────────────┤
│ 2023-06-03 │ 1.8602802623636363 │             -3.348663885 │   0.02505354445454546 │  -0.6577259409999999 │  0.004557000000000001 │                    0.0 │         99.97265625 │
│ 2023-06-04 │ 3.9927396138333333 │      -1.9052026112499998 │  -0.18090852095833332 │  -0.5052

In [14]:
con.close()